### Importing Libraries:

In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta
from statsmodels.tsa.stattools import adfuller
import requests
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
from datetime import datetime, timedelta
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from numpy import log
from itertools import combinations
from plotly.subplots import make_subplots


### Initialization:

In [ ]:
import requests
import pandas as pd
import yfinance as yf

# --------------------- CoinMarketCap: get top coins ---------------------
API_KEY = "13e10784c3884bb8b374a661da8b4631"  # keep this secret in real projects
url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"
headers = {"Accepts": "application/json", "X-CMC_PRO_API_KEY": API_KEY}
params = {"start": "1", "limit": "70", "convert": "USD"}


# request the top coins by market cap
response = requests.get(url, headers=headers, params=params)
data = response.json()["data"]
# build a small DataFrame with symbols only
top70_df = pd.DataFrame([{"symbol": coin["symbol"]} for coin in data])

#  backtest / history parameters
# Backtest start and end (tz-aware UTC timestamps)
BACKTEST_START = pd.Timestamp("2025-01-01", tz="UTC")
BACKTEST_END   = pd.Timestamp("2025-12-31", tz="UTC")
HISTORY_START  = "2023-01-01"   # earliest date to fetch history (ensures enough pre-backtest data)
TARGET_N = 50                   # target number of assets to select

# --------------------- Phase 1: Universe selection by EMA200 ratio ---------------------
selected = []   # list of selected tickers (pass EMA filter)
main_dict = {}  # store full historical df for each selected ticker (used later for backtest)
crypto_pointer = 0

# iterate through top coins until we select TARGET_N assets or exhaust the list
while len(selected) < TARGET_N and crypto_pointer < len(top70_df):
    symbol = top70_df.iloc[crypto_pointer]["symbol"]
    crypto_pointer += 1
    ticker = symbol + "-USD"

    # fetch daily historical OHLC data (include time through backtest end)
    hist = yf.Ticker(ticker).history(
        interval="1d",
        start=HISTORY_START,
        end=(BACKTEST_END + pd.Timedelta(days=1)).strftime("%Y-%m-%d")
    )

    # skip tickers with no usable price data
    if hist.empty or hist["Close"].isna().all():
        continue

    # keep only the needed columns and convert index to a timestamp column
    df = hist[["Open", "High", "Low", "Close"]].reset_index()
    df.rename(columns={"Date": "timestamp"}, inplace=True)
    df["timestamp"] = pd.to_datetime(df["timestamp"])

    # ensure all timestamps are timezone-aware and in UTC to avoid tz comparison errors
    if df["timestamp"].dt.tz is None:
        df["timestamp"] = df["timestamp"].dt.tz_localize("UTC")
    else:
        df["timestamp"] = df["timestamp"].dt.tz_convert("UTC")

    # build the pre-backtest slice (use only data strictly before BACKTEST_START)
    pre_df = df.loc[df["timestamp"] < BACKTEST_START].copy()
    # require at least 200 historical candles to compute a reliable EMA200
    if len(pre_df) < 200:
        continue

    # compute EMA200 on pre-backtest data only (avoids lookahead)
    pre_df.loc[:, "EMA200"] = pre_df["Close"].ewm(span=200, adjust=False).mean()

    # compute the fraction of pre-backtest closes that are above EMA200
    ratio_above = (pre_df["Close"] > pre_df["EMA200"]).mean()

    # apply the >50% rule: keep the ticker only if more than half of closes were above EMA200
    if ratio_above <= 0.5:
        continue

    # ticker passed the EMA filter -> store full df (we will slice BACKTEST range at backtest time)
    selected.append(ticker)
    main_dict[ticker] = df

# print selected tickers summary
print("Selected count:", len(selected))
print(selected)


# --------------------- ATR-based grid spacing ---------------------
ATR_PERIOD = 14
GRID_MULTIPLIER = 1.0   # spacing = GRID_MULTIPLIER * ATR
MIN_HISTORY_FOR_ATR = ATR_PERIOD

spacing_meta = {}  # will hold last_atr, spacing and spacing% for each ticker

def compute_atr_series(df, period=14):
    """
    Compute ATR series for a DataFrame with 'High','Low','Close'.
    Returns a pandas Series of the ATR (NaN for the first `period-1` rows).
    """
    high = df["High"]
    low = df["Low"]
    close = df["Close"]
    prev_close = close.shift(1)

    # True Range components and the TR per row
    tr = pd.concat([
        (high - low).abs(),
        (high - prev_close).abs(),
        (low - prev_close).abs()
    ], axis=1).max(axis=1)

    # ATR as a simple rolling mean of TR (min_periods=period => first valid after `period` rows)
    atr = tr.rolling(window=period, min_periods=period).mean()
    return atr



# compute ATR and derive grid spacing for each selected ticker
for ticker, df_all in main_dict.items():
    df = df_all.copy()
    df["timestamp"] = pd.to_datetime(df["timestamp"])  # ensure datetime

    # use only data before the backtest start to compute ATR (no lookahead)
    pre_df = df.loc[df["timestamp"] < BACKTEST_START].copy()
    if len(pre_df) < MIN_HISTORY_FOR_ATR:
        # not enough data to compute ATR(period)
        continue

    # compute ATR series on pre-backtest data
    pre_df.loc[:, "ATR"] = compute_atr_series(pre_df, period=ATR_PERIOD)

    # take the last ATR value as representative recent volatility
    last_atr = pre_df["ATR"].iloc[-1]
    if pd.isna(last_atr):
        # ATR may be NaN if there weren't enough TR values; skip in that case
        continue

    # grid spacing in price units (e.g., USD)
    grid_spacing = GRID_MULTIPLIER * last_atr

    # also compute spacing as a percent of last close for convenience
    last_close = pre_df["Close"].iloc[-1]
    grid_spacing_pct = grid_spacing / last_close if last_close != 0 else None

    spacing_meta[ticker] = {
        "last_atr": float(last_atr),
        "grid_spacing": float(grid_spacing),
        "grid_spacing_pct": float(grid_spacing_pct) if grid_spacing_pct is not None else None,
        "last_close": float(last_close)
    }

# print spacing summary for each ticker
for t, meta in spacing_meta.items():
    print(f"{t}: ATR={meta['last_atr']:.4f}, spacing={meta['grid_spacing']:.4f}, spacing%={meta['grid_spacing_pct']*100:.2f}%")


$PEPE-USD: possibly delisted; no price data found  (1d 2023-01-01 -> 2026-01-01)
$TAO-USD: possibly delisted; no price data found  (1d 2023-01-01 -> 2026-01-01)
$MYX-USD: possibly delisted; no price data found  (1d 2023-01-01 -> 2026-01-01)


Selected count: 40
['BTC-USD', 'ETH-USD', 'USDT-USD', 'XRP-USD', 'BNB-USD', 'SOL-USD', 'USDC-USD', 'TRX-USD', 'DOGE-USD', 'ADA-USD', 'BCH-USD', 'LINK-USD', 'LEO-USD', 'XLM-USD', 'XMR-USD', 'USDe-USD', 'AVAX-USD', 'HBAR-USD', 'SHIB-USD', 'DAI-USD', 'TON-USD', 'CRO-USD', 'MNT-USD', 'AAVE-USD', 'BGB-USD', 'OKB-USD', 'NEAR-USD', 'ETC-USD', 'ICP-USD', 'XAUt-USD', 'WLD-USD', 'PAXG-USD', 'KCS-USD', 'ONDO-USD', 'KAS-USD', 'RENDER-USD', 'VET-USD', 'BONK-USD', 'XDC-USD', 'USDD-USD']
BTC-USD: ATR=4136.0413, spacing=4136.0413, spacing%=4.43%
ETH-USD: ATR=202.1230, spacing=202.1230, spacing%=6.07%
USDT-USD: ATR=0.0021, spacing=0.0021, spacing%=0.21%
XRP-USD: ATR=0.1644, spacing=0.1644, spacing%=7.90%
BNB-USD: ATR=34.3478, spacing=34.3478, spacing%=4.90%
SOL-USD: ATR=14.3912, spacing=14.3912, spacing%=7.60%
USDC-USD: ATR=0.0017, spacing=0.0017, spacing%=0.17%
TRX-USD: ATR=0.0120, spacing=0.0120, spacing%=4.72%
DOGE-USD: ATR=0.0279, spacing=0.0279, spacing%=8.85%
ADA-USD: ATR=0.0746, spacing=0.0746, 

### Grid Configuration:

In [ ]:
def create_simple_grid_levels(current_price, grid_spacing, position_size, n_levels=10):
    """
    Create a simple buy-only grid below the current price.
    """

    buy_levels = np.array([
        current_price - (i + 1) * grid_spacing
        for i in range(n_levels)
    ])

    sell_levels = buy_levels + grid_spacing

    buy_to_sell_mapping = {
        buy_levels[i]: sell_levels[i]
        for i in range(n_levels)
    }

    return {
        "buy_levels": buy_levels,
        "sell_levels": sell_levels,
        "position_size": position_size,
        "grid_spacing": grid_spacing,
        "buy_to_sell_mapping": buy_to_sell_mapping,
        "available_sells": set(sell_levels)
    }



def compute_martingale_sizes(base_size, multiplier, n_levels):
    """
    Compute exponentially increasing position sizes for grid levels.
    """
    sizes = []

    for i in range(n_levels):
        size = base_size * (multiplier ** i)
        sizes.append(size)

    return sizes



# Just for testing the compute_average_price method and it is not used for strategy implementation.
filled_buys = [
    {"price": 100, "size": 1},
    {"price": 90,  "size": 2},
    {"price": 80,  "size": 4},
]


def compute_average_price(filled_buys):
    """
    Compute weighted average buy price of the active grid.
    """
    total_cost = 0.0
    total_size = 0.0

    for order in filled_buys:
        total_cost += order["price"] * order["size"]
        total_size += order["size"]

    if total_size == 0:
        return None

    return total_cost / total_size


avg_price = compute_average_price(filled_buys)
print(avg_price)





85.71428571428571
